In [0]:
import pyspark
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import DataFrameWriter

In [0]:
spark.conf.set("fs.azure.account.key.ksrblobst.dfs.core.windows.net", "rpuQnbZpsJL3zRK2/BHjnJmXTmhxwrTRisZz6ieX3w0VbRJb2rIPJDcxc7CbNQtG01rHfJAGhM7O+ASteaBTcQ==")

In [0]:
display(dbutils.fs.ls("abfss://ksroot@ksrblobst.dfs.core.windows.net"))

path,name,size,modificationTime
abfss://ksroot@ksrblobst.dfs.core.windows.net/2393276-Bike_Data.csv,2393276-Bike_Data.csv,6956530,1708345249000
abfss://ksroot@ksrblobst.dfs.core.windows.net/investment_banking.csv,investment_banking.csv,57244620,1708342919000


In [0]:
data = spark.read.csv("abfss://ksroot@ksrblobst.dfs.core.windows.net/investment_banking.csv", header=True, inferSchema=True)

In [0]:
#data.show(3)


In [0]:
#display(len(data.columns), data.count())

In [0]:
#data.printSchema()

In [0]:
#data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]
#   ).show()

In [0]:
data.select([round(((count(when(isnan(c) | col(c).isNull(),c))/data.count())*100),2).alias(c) for c in data.columns])

DataFrame[broker_id: double, city: double, broker_type: double, fund_category: double, email_opened: double, webex_meet: double, sales_call: double, firm_sales: double, global_sales: double]

In [0]:
#columns to be dropped (webex_meet,sales_call) because they are more then 75 % null values
# for column (email_opened) the null values will be replaced with (N/A)

In [0]:
data = data.drop(col("webex_meet"),col("sales_call"))

In [0]:
#column email_opened has 57.7 % null we can keep and replace the null with NA

In [0]:
columns = ["email_opened"]

In [0]:
data = data.fillna("N/A", subset = columns)

In [0]:
#column city contains state and city, we will split in 2 columns

In [0]:
data = data.withColumn('state', split(data['city'], ',').getItem(1))


In [0]:
data = data.withColumn('City', split(data['city'], ',').getItem(0))

In [0]:
#now check these columns toroughly

In [0]:
#data.show(10)

+---------+-------------+-------------------+--------------------+------------+----------+------------+-----+
|broker_id|         City|        broker_type|       fund_category|email_opened|firm_sales|global_sales|state|
+---------+-------------+-------------------+--------------------+------------+----------+------------+-----+
|   BRXX-1|   PLANTATION|Inter-dealer broker|Emerging-Markets ...|        NULL|    174.62|      174.62|   FL|
|   BRXX-1|     BRANFORD|Inter-dealer broker|           Utilities|        NULL|       0.0|         0.0|   CT|
|   BRXX-1|    JONESBORO|Inter-dealer broker|Intermediate Gove...|        NULL|       0.0|         0.0|   GA|
|   BRXX-2|       VIENNA|Inter-dealer broker|Intermediate Gove...|           Y|       0.0|     30709.0|   VA|
|   BRXX-3|CHAGRIN FALLS|full-service broker|    Target-Date 2050|           Y|       0.0|         0.0|   OH|
|   BRXX-2|     CARLSBAD|Inter-dealer broker|       Multicurrency|           Y|       0.0|         0.0|   CA|
|   BRXX-3

In [0]:
#data.filter(length(col("state"))>2 ).distinct().show()

+---------+-------------+-------------------+--------------------+------------+----------+----------+----------+------------+-----+
|broker_id|         City|        broker_type|       fund_category|email_opened|webex_meet|sales_call|firm_sales|global_sales|state|
+---------+-------------+-------------------+--------------------+------------+----------+----------+----------+------------+-----+
|   BRXX-3|BEVERLY HILLS|full-service broker|Inflation-Protect...|           Y|         Y|         Y|       0.0|      2143.6|   CA|
|   BRXX-3|       VIENNA|full-service broker|         Large Value|           Y|         Y|         Y| 175211.54|  2126109.36|   VA|
|   BRXX-3|        OGDEN|full-service broker|        Small Growth|           Y|         Y|         Y|       0.0|     6595.17|   UT|
|   BRXX-3|       RESTON|full-service broker|Equity Precious M...|           Y|         Y|         Y|       0.0|         0.0|   VA|
|   BRXX-3|SAN FRANCISCO|full-service broker|     High Yield Muni|          

In [0]:
 #data.select(col("state"), length(col("state"))).filter(length(col("state"))>3).distinct().show()

+-----+-------------+
|state|length(state)|
+-----+-------------+
+-----+-------------+



In [0]:
# there is extra space and extra comma in the state column, we need to remove it.

In [0]:
data=data.withColumn("state", regexp_replace(col("state"), " ", ""))

In [0]:
 #data.select(col("state"), length(col("state"))).filter(length(col("state"))>2).distinct().show()

+-----+-------------+
|state|length(state)|
+-----+-------------+
+-----+-------------+



In [0]:
#verify in each string type columns if any value is numerical

In [0]:
#data.select("state", col("state").cast("int").isNotNull().alias("Value ")).show(10)

In [0]:
#data.select("city", col("city").cast("int").isNotNull().alias("Value ")).show(10)

+-------------+------+
|         city|Value |
+-------------+------+
|   PLANTATION| false|
|     BRANFORD| false|
|    JONESBORO| false|
|       VIENNA| false|
|CHAGRIN FALLS| false|
|     CARLSBAD| false|
|     BELLEVUE| false|
| OWINGS MILLS| false|
|       SUMMIT| false|
|   CHARLESTON| false|
+-------------+------+
only showing top 10 rows



In [0]:
#data.select("broker_id", col("broker_id").cast("int").isNotNull().alias("Value ")).show(10)

In [0]:
#data.select("broker_type", col("broker_type").cast("int").isNotNull().alias("Value ")).show(10)

In [0]:
#data.select("fund_category", col("fund_category").cast("int").isNotNull().alias("Value ")).show(10)

+--------------------+------+
|       fund_category|Value |
+--------------------+------+
|Emerging-Markets ...| false|
|           Utilities| false|
|Intermediate Gove...| false|
|Intermediate Gove...| false|
|    Target-Date 2050| false|
|       Multicurrency| false|
|Target-Date Retir...| false|
|   World Large Stock| false|
|Intermediate-Term...| false|
|        Convertibles| false|
+--------------------+------+
only showing top 10 rows



In [0]:
#data is clean

In [0]:
#data.show(10)

+---------+-------------+-------------------+--------------------+------------+----------+------------+-----+
|broker_id|         City|        broker_type|       fund_category|email_opened|firm_sales|global_sales|state|
+---------+-------------+-------------------+--------------------+------------+----------+------------+-----+
|   BRXX-1|   PLANTATION|Inter-dealer broker|Emerging-Markets ...|         N/A|    174.62|      174.62|   FL|
|   BRXX-1|     BRANFORD|Inter-dealer broker|           Utilities|         N/A|       0.0|         0.0|   CT|
|   BRXX-1|    JONESBORO|Inter-dealer broker|Intermediate Gove...|         N/A|       0.0|         0.0|   GA|
|   BRXX-2|       VIENNA|Inter-dealer broker|Intermediate Gove...|           Y|       0.0|     30709.0|   VA|
|   BRXX-3|CHAGRIN FALLS|full-service broker|    Target-Date 2050|           Y|       0.0|         0.0|   OH|
|   BRXX-2|     CARLSBAD|Inter-dealer broker|       Multicurrency|           Y|       0.0|         0.0|   CA|
|   BRXX-3

In [0]:
#data.printSchema()

root
 |-- broker_id: string (nullable = true)
 |-- City: string (nullable = true)
 |-- broker_type: string (nullable = true)
 |-- fund_category: string (nullable = true)
 |-- email_opened: string (nullable = false)
 |-- firm_sales: double (nullable = true)
 |-- global_sales: double (nullable = true)
 |-- state: string (nullable = true)



In [0]:
#write final clean data present in dataframe"data" into the database

In [0]:
# we need to configure the database credentials here
jdbcHostName = "ksrsqlserver.database.windows.net"
jdbcPort = 1433
jdbcDatabase = "ksrdb1"
dbproperties = {
    "user": "mouni",
    "password": "Rylee23$"
}

In [0]:
serverurl = "jdbc:sqlserver://{0}:{1};database={2}".format(jdbcHostName,jdbcPort,jdbcDatabase)

In [0]:
print(serverurl)

jdbc:sqlserver://ksrsqlserver.database.windows.net:1433;database=ksrdb1


In [0]:
# write back the data into sqlserver
output = DataFrameWriter(data)
output.jdbc(url = serverurl, table = "investment_bank_clen_tbl", mode = "overWrite", properties = dbproperties )